In [1]:
import os
import shutil
import json
import polus.plugins
from polus.plugins import plugins
from pathlib import Path
from pprint import pprint

In [3]:
manifest_path = Path('__file__').with_name('manifests.json')
with open(manifest_path, 'r') as fr:
    manifest_urls = json.load(fr)
    fr.close()
    
pprint(manifest_urls)

['https://raw.githubusercontent.com/Nicholas-Schaub/polus-plugins/fix/flatfield/regression/polus-basic-flatfield-correction-plugin/plugin.json',
 'https://raw.githubusercontent.com/Nicholas-Schaub/polus-plugins/fix/flatfield/transforms/images/polus-apply-flatfield-plugin/plugin.json',
 'https://raw.githubusercontent.com/bthoughton/polus-plugins/fix/montage/transforms/images/polus-montage-plugin/plugin.json',
 'https://raw.githubusercontent.com/PolusAI/polus-plugins/dev/transforms/images/polus-image-assembler-plugin/plugin.json',
 'https://raw.githubusercontent.com/PolusAI/polus-plugins/master/visualization/polus-precompute-slide-plugin/plugin.json']


In [4]:
for url in manifest_urls:
    polus.plugins.submit_plugin(url, refresh=True)
    
print(plugins.list)

['ApplyFlatfield', 'BasicFlatfieldCorrectionPlugin', 'ImageAssembler', 'Montage', 'PolusPrecomputeSlidePlugin', 'PolusPrecomputeVolumePlugin']


In [5]:
inp_dir = Path('/home/ec2-user/subset/intensity/')
assert inp_dir.exists(), "The input directory does not exist"

inp_files = [file.name for file in inp_dir.iterdir()]
inp_files.sort()
print('The number of input images is {}'.format(len(inp_files)))
print('\nThe first 5 images:')
pprint(inp_files[:5])
print('\nThe last 5 images:')
pprint(inp_files[-5:])

file_pattern = 'p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif'

The number of input images is 6908

The first 5 images:
['p00_x01_y01_wx0_wy0_c1.ome.tif',
 'p00_x01_y01_wx0_wy0_c2.ome.tif',
 'p00_x01_y01_wx0_wy1_c1.ome.tif',
 'p00_x01_y01_wx0_wy1_c2.ome.tif',
 'p00_x01_y01_wx0_wy2_c1.ome.tif']

The last 5 images:
['p00_x24_y16_wx2_wy0_c2.ome.tif',
 'p00_x24_y16_wx2_wy1_c1.ome.tif',
 'p00_x24_y16_wx2_wy1_c2.ome.tif',
 'p00_x24_y16_wx2_wy2_c1.ome.tif',
 'p00_x24_y16_wx2_wy2_c2.ome.tif']


In [6]:
out_dir = {}
parent_out_dir = Path('/home/ec2-user/subset/visualization-out')
workflow = [
    'ApplyFlatfield', 'BasicFlatfieldCorrectionPlugin', 'ImageAssembler', 'Montage', 'PolusPrecomputeSlidePlugin'
    ]

if not parent_out_dir.exists():
    os.mkdir(parent_out_dir)

for name in workflow:
    out_dir[name] = parent_out_dir.joinpath(name)
    if out_dir[name].exists():
        continue
        ### CAUTION THIS REMOVES OLD OUT DIRECTORIES
        shutil.rmtree(out_dir[name])
    elif name :
        os.mkdir(out_dir[name])
    
pprint(out_dir)

{'ApplyFlatfield': PosixPath('/home/ec2-user/subset/visualization-out/ApplyFlatfield'),
 'BasicFlatfieldCorrectionPlugin': PosixPath('/home/ec2-user/subset/visualization-out/BasicFlatfieldCorrectionPlugin'),
 'ImageAssembler': PosixPath('/home/ec2-user/subset/visualization-out/ImageAssembler'),
 'Montage': PosixPath('/home/ec2-user/subset/visualization-out/Montage'),
 'PolusPrecomputeSlidePlugin': PosixPath('/home/ec2-user/subset/visualization-out/PolusPrecomputeSlidePlugin')}


In [7]:
flatfield_corr = plugins.BasicFlatfieldCorrectionPlugin
apply_flatfield = plugins.ApplyFlatfield
montage = plugins.Montage
assembler = plugins.ImageAssembler
precompute = plugins.PolusPrecomputeSlidePlugin

In [8]:
[inp.name for inp in flatfield_corr.inputs + flatfield_corr.outputs]

['inpDir', 'filePattern', 'darkfield', 'photobleach', 'groupBy', 'outDir']

In [9]:
flatfield_corr.inpDir = inp_dir
flatfield_corr.filePattern = file_pattern
flatfield_corr.darkfield = True
flatfield_corr.photobleach = False
flatfield_corr.groupBy = 'xytp'
flatfield_corr.outDir = str(out_dir['BasicFlatfieldCorrectionPlugin'])

for inp in flatfield_corr.inputs + flatfield_corr.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = /home/ec2-user/subset/intensity
filePattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
darkfield = True
photobleach = False
groupBy = xytp
outDir = /home/ec2-user/subset/visualization-out/BasicFlatfieldCorrectionPlugin


In [22]:
flatfield_corr.run(gpus=None)

16-Sep-22 14:43:53 - polus.plugins - INFO     - Running container without GPU. BasicFlatfieldCorrectionPlugin version 1.2.8
16-Sep-22 14:43:54 - main - INFO - Parsing arguments...
16-Sep-22 14:43:54 - main - INFO - input_dir = /data/inputs/input0
16-Sep-22 14:43:54 - main - INFO - get_darkfield = True
16-Sep-22 14:43:54 - main - INFO - get_photobleach = False
16-Sep-22 14:43:54 - main - INFO - file_pattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
16-Sep-22 14:43:54 - main - INFO - output_dir = /data/outputs/output0/images
16-Sep-22 14:43:55 - proc 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif: Loading and sorting images...
16-Sep-22 14:44:25 - basic - INFO - Processes   0.00% complete
16-Sep-22 14:44:29 - proc 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif: Beginning flatfield estimation
16-Sep-22 14:44:55 - basic - INFO - Processes   0.00% complete
16-Sep-22 14:45:25 - basic - INFO - Processes   0.00% complete
16-Sep-22 14:45:40 - proc 1 - INFO - p00_x(01-

In [10]:
[inp.name for inp in apply_flatfield.inputs + apply_flatfield.outputs]

['darkPattern',
 'ffDir',
 'brightPattern',
 'imgDir',
 'imgPattern',
 'photoPattern',
 'outDir']

In [11]:
apply_flatfield.imgDir = inp_dir
apply_flatfield.imgPattern = file_pattern
apply_flatfield.ffDir = out_dir['BasicFlatfieldCorrectionPlugin']
apply_flatfield.brightPattern = 'p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif'
apply_flatfield.darkPattern = 'p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif'
apply_flatfield.outDir = out_dir['ApplyFlatfield']

for inp in apply_flatfield.inputs + apply_flatfield.outputs:
    print('{} = {}'.format(inp.name, inp.value))

darkPattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif
ffDir = /home/ec2-user/subset/visualization-out/BasicFlatfieldCorrectionPlugin
brightPattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif
imgDir = /home/ec2-user/subset/intensity
imgPattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
photoPattern = None
outDir = /home/ec2-user/subset/visualization-out/ApplyFlatfield


In [32]:
apply_flatfield.run(gpus=None)

16-Sep-22 15:43:51 - polus.plugins - INFO     - Running container without GPU. ApplyFlatfield version 1.0.6


Unable to find image 'labshare/polus-apply-flatfield-plugin:1.0.6' locally
1.0.6: Pulling from labshare/polus-apply-flatfield-plugin
c549ccf8d472: Already exists
e920f29c01be: Already exists
7ad2a8fabcac: Already exists
b3bf2315b004: Already exists
46e2d4d94b50: Already exists
354e28b2d647: Already exists
8126bbab890d: Pulling fs layer
0abb870dfc7a: Pulling fs layer
48c5046985f3: Pulling fs layer
0abb870dfc7a: Verifying Checksum
0abb870dfc7a: Download complete
8126bbab890d: Verifying Checksum
8126bbab890d: Download complete
48c5046985f3: Verifying Checksum
48c5046985f3: Download complete
8126bbab890d: Pull complete
0abb870dfc7a: Pull complete
48c5046985f3: Pull complete
Digest: sha256:0dfe9e979f1b168589ed2fa3ba1db78999ba1f28c98592250a22a9bfde9098d6
Status: Downloaded newer image for labshare/polus-apply-flatfield-plugin:1.0.6
docker.io/labshare/polus-apply-flatfield-plugin:1.0.6


16-Sep-22 15:43:53 - main     - INFO     - Parsing arguments...
16-Sep-22 15:43:53 - main     - INFO     - darkPattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif
16-Sep-22 15:43:53 - main     - INFO     - ffDir = /data/inputs/input0/images
16-Sep-22 15:43:53 - main     - INFO     - brightPattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif
16-Sep-22 15:43:53 - main     - INFO     - imgDir = /data/inputs/input1
16-Sep-22 15:43:53 - main     - INFO     - imgPattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
16-Sep-22 15:43:53 - main     - INFO     - photoPattern = None
16-Sep-22 15:43:53 - main     - INFO     - outDir = /data/outputs/output0
16-Sep-22 15:44:04 - unshade 1 - INFO     -   2.29% complete
16-Sep-22 15:44:14 - unshade 1 - INFO     -   4.31% complete
16-Sep-22 15:44:23 - main     - INFO     - Processes   0.00% complete
16-Sep-22 15:44:24 - unshade 1 - INFO     -   6.46% complete
16-Sep-22 15:44:34 - unshade 1 - INFO     -   8.14% complete


In [12]:
[inp.name for inp in montage.inputs + montage.outputs]

['filePattern',
 'inpDir',
 'layout',
 'imageSpacing',
 'gridSpacing',
 'flipAxis',
 'outDir']

In [13]:
montage.inpDir = out_dir['ApplyFlatfield']
montage.filePattern = 'p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif' # Montage only channel 1
montage.layout = 'tp, xy'
montage.imageSpacing = 1
montage.gridSpacing = 20
montage.flipAxis = 'p' # this has to be specified or the plugin will return error this is a bug
montage.outDir = out_dir['Montage']

for inp in montage.inputs + montage.outputs:
    print('{} = {}'.format(inp.name, inp.value))

filePattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
inpDir = /home/ec2-user/subset/visualization-out/ApplyFlatfield
layout = tp, xy
imageSpacing = 1
gridSpacing = 20
flipAxis = p
outDir = /home/ec2-user/subset/visualization-out/Montage


In [14]:
montage.run(gpus=None)

20-Sep-22 20:03:15 - polus.plugins - INFO     - Running container without GPU. Montage version 0.4.0
20-Sep-22 20:03:16 - main - INFO - Parsing arguments...
20-Sep-22 20:03:16 - main - INFO - filePattern = p00_x{xx}_y{yy}_wx{t}_wy{p}_c{c}.ome.tif
20-Sep-22 20:03:16 - main - INFO - inpDir = /data/inputs/input0
20-Sep-22 20:03:16 - main - INFO - layout = tp, xy
20-Sep-22 20:03:16 - main - INFO - flipAxis = p
20-Sep-22 20:03:16 - main - INFO - outDir = /data/outputs/output0
20-Sep-22 20:03:16 - main - INFO - image_spacing = 1
20-Sep-22 20:03:16 - main - INFO - grid_spacing = 20
20-Sep-22 20:03:16 - main - INFO - Parsing the file pattern...
20-Sep-22 20:03:16 - main - INFO - Parsing the layout...
20-Sep-22 20:03:16 - main - INFO - Get the size of every image...
20-Sep-22 20:03:24 - main - INFO - Grid size for layer (tp): [3, 3]
20-Sep-22 20:03:24 - main - INFO - Grid size for layer (xy): [24, 16]
20-Sep-22 20:03:24 - main - INFO - Final image size in pixels: [78312, 52208]
20-Sep-22 20:03:

In [15]:
vector_paths = [p for p in out_dir['Montage'].iterdir()]
pprint(vector_paths)

[PosixPath('/home/ec2-user/subset/visualization-out/Montage/p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.txt'),
 PosixPath('/home/ec2-user/subset/visualization-out/Montage/p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c2.txt')]


In [16]:
vector_c1 = []

with open(vector_paths[0], 'r') as fr:
    for line in fr:
        s = line.strip()
        vector_c1.append(s)
    fr.close()
    
# pprint(vector)
pprint(vector_c1[:5])
print(len(vector_c1))

['file: p00_x01_y01_wx0_wy0_c1.ome.tif; corr: 0; position: (0, 2162); grid: '
 '(0, 2);',
 'file: p00_x01_y01_wx0_wy1_c1.ome.tif; corr: 0; position: (0, 1081); grid: '
 '(0, 1);',
 'file: p00_x01_y01_wx0_wy2_c1.ome.tif; corr: 0; position: (0, 0); grid: (0, '
 '0);',
 'file: p00_x02_y01_wx0_wy0_c1.ome.tif; corr: 0; position: (3263, 2162); grid: '
 '(3, 2);',
 'file: p00_x02_y01_wx0_wy1_c1.ome.tif; corr: 0; position: (3263, 1081); grid: '
 '(3, 1);']
3454


In [17]:
for i, path in enumerate(vector_paths):
    os.rename(path, path.with_name('img-global-positions-{}.txt'.format(i+1)))
    
vector_paths = [p for p in out_dir['Montage'].iterdir()]
pprint(vector_paths)

[PosixPath('/home/ec2-user/subset/visualization-out/Montage/img-global-positions-1.txt'),
 PosixPath('/home/ec2-user/subset/visualization-out/Montage/img-global-positions-2.txt')]


In [18]:
[inp.name for inp in assembler.inputs + assembler.outputs]

['stitchPath', 'imgPath', 'timesliceNaming', 'outDir']

In [19]:
assembler.imgPath = out_dir['ApplyFlatfield']
assembler.stitchPath = out_dir['Montage']
assembler.outDir = out_dir['ImageAssembler']
assembler.timesliceNaming = True

for inp in assembler.inputs + assembler.outputs:
    print('{} = {}'.format(inp.name, inp.value))

stitchPath = /home/ec2-user/subset/visualization-out/Montage
imgPath = /home/ec2-user/subset/visualization-out/ApplyFlatfield
timesliceNaming = True
outDir = /home/ec2-user/subset/visualization-out/ImageAssembler


In [20]:
assembler.run(gpus=None)

20-Sep-22 20:04:03 - polus.plugins - INFO     - Running container without GPU. ImageAssembler version 1.1.4
20-Sep-22 20:04:04 - main - INFO - imgPath: /data/inputs/input1
20-Sep-22 20:04:04 - main - INFO - outDir: /data/outputs/output0
20-Sep-22 20:04:04 - main - INFO - timesliceNaming: False
20-Sep-22 20:04:04 - main - INFO - stitchPath: /data/inputs/input0
20-Sep-22 20:04:04 - main - INFO - Inferred file pattern: p00_x{rr}_y{tt}_wx{c}_wy{z}_c{y}.ome.tif
20-Sep-22 20:04:05 - asmbl 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif: Inferred output file name from vector.
20-Sep-22 20:04:05 - asmbl 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif: Begin assembly
20-Sep-22 20:04:05 - asmbl 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif: Assembling Z position : 0
20-Sep-22 20:04:15 - asmbl 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif:   2.86% complete
20-Sep-22 20:04:25 - asmbl 1 - INFO - p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif:   

In [21]:
[img.name for img in out_dir['ImageAssembler'].iterdir()]

['p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c1.ome.tif',
 'p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c2.ome.tif']

In [22]:
[inp.name for inp in precompute.inputs + precompute.outputs]

['inpDir', 'pyramidType', 'imageType', 'filePattern', 'outDir']

In [23]:
precompute.inpDir = out_dir['ImageAssembler']
precompute.imageType = 'image'
precompute.filePattern = 'p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}.ome.tif'
precompute.pyramidType = 'Zarr'
precompute.outDir = out_dir['PolusPrecomputeSlidePlugin']

for inp in precompute.inputs + precompute.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = /home/ec2-user/subset/visualization-out/ImageAssembler
pyramidType = pyramidType.Zarr
imageType = imageType.image
filePattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}.ome.tif
outDir = /home/ec2-user/subset/visualization-out/PolusPrecomputeSlidePlugin


In [24]:
precompute.run(gpus=None)

20-Sep-22 20:13:30 - polus.plugins - INFO     - Running container without GPU. PolusPrecomputeSlidePlugin version 1.4.5
20-Sep-22 20:13:32 - main - INFO - Parsing arguments...
20-Sep-22 20:13:32 - main - INFO - input_dir = /data/inputs/input0
20-Sep-22 20:13:32 - main - INFO - output_dir = /data/outputs/output0
20-Sep-22 20:13:32 - main - INFO - pyramid_type = Zarr
20-Sep-22 20:13:32 - main - INFO - image_type = image
20-Sep-22 20:13:32 - main - INFO - file_pattern = p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c{c}.ome.tif
20-Sep-22 20:13:32 - main - INFO - max concurrent processes = 2
20-Sep-22 20:13:32 - main - INFO - Stacking channels by c-dimension for Zarr format
20-Sep-22 20:13:42 - proc 1 - INFO - /data/outputs/output0/p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c(1-2) - 0:   0.00% complete
20-Sep-22 20:13:42 - proc 2 - INFO - /data/outputs/output0/p00_x(01-24)_y(01-16)_wx(0-2)_wy(0-2)_c(1-2) - 1:   0.00% complete
20-Sep-22 20:13:52 - proc 1 - INFO - /data/outputs/output0/p00_x(01-24)_y(01-